# Histogram Equalization
Joshua Stough
DIP

Showing histogram equalization both through application of an intensity transform, and empirically with the histogram.
Shown on the Gaussian normal color random image.

In [1]:
%matplotlib widget
# %matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import numpy.random as random

### Make Gausisan and Uniform Random Images

See how the uniform has better contrast, see what that means
in the histograms

In [2]:
IU = np.concatenate([random.rand(100,100,1) for x in range(3)], axis = 2)
IN = np.concatenate([random.randn(100,100,1) for x in range(3)], axis = 2)
IN = IN - IN.ravel().min() #Make the minimum 0
IN = IN/IN.ravel().max()  #Make max 1

In [3]:
f, axarr = plt.subplots(1,2, figsize=(8, 3))
axarr[0].imshow(IU)
axarr[0].set_title('Uniform Random Image')
axarr[1].imshow(IN)
axarr[1].set_title('Normal Random Image')

f, ax = plt.subplots(1,2, figsize=(8,3))
ax[0].hist(IU.ravel(), bins=256);
ax[1].hist(IN.ravel(), bins=256);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Visualize the two images in the color cube.
Some complicated code you should understand. 
Notice how the uniform distribution is better spread in the color cube.

In [4]:
'''
visCube(I): visualize the image I within the color cube.
'''
def visCube(I, numpoints = 5000):

    fig = plt.figure(figsize=(5,4))
    ax = fig.add_subplot(111, projection='3d')

    #X is the N*M x 3 version of the image.
    X = np.concatenate([np.expand_dims(Ichan, axis = 1) for Ichan in
                        [I[...,0].ravel(), I[...,1].ravel(), I[...,2].ravel()]], axis = 1)

    #https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.choice.html
    randomInds = np.random.choice(np.arange(X.shape[0]), numpoints, replace=False)
    
    # point colors
    point_colors = X[randomInds, :]
    if point_colors.max() > 1:
        point_colors = point_colors-point_colors.min()
        point_colors = point_colors/point_colors.max()

    #Now plot those pixels in the 3d space.
    ax.scatter(X[randomInds,0], X[randomInds,1], X[randomInds,2], 
               c=point_colors, depthshade=False)

    #Label the axes.
    ax.set_xlabel('Red')
    ax.set_ylabel('Green')
    ax.set_zlabel('Blue')

In [5]:
visCube(IU)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
visCube(IN)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Build a parametric function that would spread the data out in IN.
We decided that would be an [S curve or sigmoid](https://en.wikipedia.org/wiki/Sigmoid_function).

$$S(x) = \frac{1}{1 + e^{-x}}$$

But we need a sigmoid that is centered around .5 and steep enough to look like an s inside [0,1].

$$S(x, x_c, s) = \frac{1}{1 + e^{-s(x-x_c)}}$$

In [7]:
x = np.arange(0,1,1/100)
f = lambda x: 1/(1+np.exp(-(10*(x-.5))))
plt.figure(figsize=(4,3))
plt.plot(x, f(x));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Apply the function to the image IN and see how it improves contrast.

In [8]:
fff, ax = plt.subplots(1,2, figsize=(6,2), sharex=True, sharey=True)

ax[0].imshow(IN)
ax[1].imshow(f(IN));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
visCube(f(IN))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### We can accomplish the same thing emperically, through the [cumulative distribution function](https://en.wikipedia.org/wiki/Cumulative_distribution_function) on the histogram

A fundamental result from probability theory, if $s = T(r)$ for $T$ continuous and differentiable:
$$p_s(r) = p_r(r)\begin{vmatrix}{\frac{dr}{ds}}\end{vmatrix}$$

The CDF measures the probability that a random variable $X$ takes on a value less than or equal to $x$:

 $$P(X \le x) = \intop_{0}^{x} p_r(w)dw$$
 
where $p_r$ is the probability distribution (read: histogram) of our random variable $X$.

In [10]:
# First get the histogram of IN, and plot with the cdf.
# normalize so they show up nicely.

freq, bins = np.histogram(IN.ravel(), bins=x)
freq = freq/freq.max()
cdf = np.cumsum(freq)
cdf = cdf/cdf.max()

f = plt.figure(figsize=(4,3))
plt.bar(bins[:-1], freq, width=.01) # width to keep the bars skinny enough.
plt.plot(bins[:-1], cdf, 'r-');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### It's coincidental that the sigmoid is the CDF of a Gaussian.  

Let's apply the function that uses this empirical cdf (coming from the histogram).


In [11]:
from scipy.interpolate import interp1d

func = interp1d(bins[:-1], cdf, fill_value='extrapolate') 
# 'extrapolate' is important here, otherwise a lot of out of bound errors.

In [12]:
fff, ax = plt.subplots(1,2, figsize=(6,2), sharex=True, sharey=True)
# fIN = np.stack([func(IN[...,chan]) for chan in range(3)], axis=-1)
ax[0].imshow(IN)
ax[1].imshow(func(IN));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [13]:
# View the two histograms
f, axarr = plt.subplots(1,1, figsize=(5, 4))

funcI = func(IN)

axarr.hist(IN.ravel(), bins=256, alpha = .6, label = 'Original', color = 'r');
axarr.hist(funcI.ravel(), bins=256, alpha = .6, label = 'Equalized', color = 'g');
axarr.legend(loc = 'upper right');
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …